## How is the dispersal rate estimation by BEAST affected by sampling area and dispersal model?

This notebook investigates how a circular **sampling area** influences the accuracy and variability of dispersal rate estimates inferred by BEAST using continuous phylogeographic diffusion models.

---

### GSpace simulation using various sampling areas

We simulate sequence data under controlled spatial population genetic conditions using GSpace. The key variable in this notebook is the **sampling radius** $r$, which defines the radius of the sampling area starting from the center of the grid.

#### Sampling strategy:

Sampling is performed in a circular region centered on the lattice midpoint. For each node:

-	A random angle $\phi \sim \mathcal{U}(0, 2\pi)$ is drawn to ensure isotropic sampling.
-	A random radius $r{\prime} \sim \mathcal{U}(0, r)$ is drawn to achieve uniform density over the area of the circle.
-	Using trigonometric transformation, polar coordinates are converted to Cartesian:
$x = x_{\text{center}} + r{\prime} \cdot \cos(\phi)$, $\quad y = y_{\text{center}} + r{\prime} \cdot \sin(\phi)$
-	The resulting floating-point coordinate is mapped to the nearest integer lattice node via Euclidean distance minimization among the four surrounding nodes.
-	Only nodes strictly within the lattice boundaries are retained.

### Workflow:

In [1]:
import shutil
from utils.gspace_utils import *
from utils.beast_utils import *
from utils.file_utils import *

g_mutation_rate=0.00001
radii = [1, 10, 50, 100]

#### 1. Set the working directory

Set the working where your GSpaceSettings.txt file will be generated. further results of the analysis will be found here as well.

In [2]:
set_tests_dir()

Moved into 'Tests' directory. Current working directory: /Users/ayoubrayaneaitallaoua/Documents/LIRMM/Pathogen_Dispersal_Rate_Estimation/Tests


#### 2. generate GSpace Settings.txt with variable radii

In [3]:
for radius in radii:
    generate_gspace_settings_circular_sample(r=radius, mutation_rate=g_mutation_rate)

GSpaceSettings_r_5.txt generated with circular sampling in .!


#### 3. run GSpace simulations

In [ ]:
for radius in radii:

        # make a dir for each radius
        os.mkdir(f"result_r_{radius}")

        # move the GSpacesettings_r_{r}.txt files into their respective directories
        shutil.move(
            f"GSpacesettings_r_{radius}.txt",
            f"result_r_{radius}/GSpacesettings_r_{radius}.txt")

        # change directories for each radius and rename the file
        os.chdir(f"result_r_{radius}")
        os.rename(f"GSpaceSettings_r_{radius}.txt","GSpaceSettings.txt")

        # run GSpace
        run_gspace(gspace_dir="../../../GSpace/build/GSpace")

        # go back to Tests dir
        os.chdir("..")

#### 4. generate BEAST XML files

In [ ]:
set_tests_dir()

for radius in radii:
    os.chdir(f"result_r_{radius}")
    generate_beast_xml(output_xml=f"r_{radius}.xml", mutation_rate=g_mutation_rate)
    os.chdir("..")

print(f"current working directory:{os.getcwd()}")

#### 5. run BEAST for each simulation

In [ ]:
print(os.getcwd())
for radius in radii:
    os.chdir(f"result_r_{radius}")
    run_beast(xml_file=f"r_{radius}.xml",beast_path="../../../BEAST/bin/beast")
    os.chdir("..")